In [93]:
import numpy as np
import os, re, csv, math, codecs
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer as t
from tqdm import tqdm
from keras.layers import merge, Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.models import *
# to visualize, and to make zero shape matrix
from attention_utils import get_activations, get_data_recurrent
from Attention import Attention
import pandas as pd
from konlpy.tag import Okt as Twitter
from selfword2vec import tokenization
from Anomaly import checkAnomaly_x_y
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from numpy import argmax
from keras import backend as K


# 1. 모델 저장시키기 
# 2. tokenizer와 konlpy morphs 호환 여부 (완성 jupyter note)
# 2'. word2vec 2가지 경우 더 추가 to embedding(final_total word2vec, twitter_translated.vec)
# 3. 변수들 설정하기


np.random.seed(3)

# NUM_WORDS, train으로 input받은 단어의 수
MAX_NB_WORDS = 20000
vocab_size = 0
EMB_DIM = 300
embeddings_index = dict()

# columns = ["ID", "Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism","sadness","surprise","trust"]
columns = ["ID","Tweet","분노","기대","혐오스러운","두려움","기쁨","사랑","낙관론","비관론","슬픔","놀라움","믿음"]




##########################################1. 데이터셋 생성하기

train_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_train.txt",sep="\t", header=None,names=columns).values
val_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_dev.txt",sep="\t", header=None,names=columns).values
test_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_test_gold.txt",sep="\t", header=None,names=columns).values

# 판다 shape
print("train_array"+ str(train_array.shape))
print("val_array"+str(val_array.shape))
print("test_array"+str(test_array.shape))

print("Reading data!")
#  x, y 분할하기
x_train = train_array[1:,1]
y_train = train_array[1:,2:]
x_val = val_array[1:,1]
y_val = val_array[1:,2:]
x_test = test_array[1:,1]
y_test = test_array[1:,2:]


print("checking Anomaly!!")
x_train, y_train = checkAnomaly_x_y(x_train,y_train)
x_val, y_val = checkAnomaly_x_y(x_val,y_val)
x_test, y_test = checkAnomaly_x_y(x_test,y_test)

print("ANomaly result!")
print("x_train.shape"+ str(x_train.shape))
print("y_train.shape"+ str(y_train.shape))
print("x_val.shape"+ str(x_val.shape))
print("y_val.shape"+ str(y_val.shape))
print("x_test.shape"+ str(x_test.shape))
print("y_test.shape"+ str(y_test.shape))
# print(type(x_train)) np.array로 변형이 필요한가
print("Finished!")

# jupyter notebook
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------

tmp = []
train_tmp = []
val_tmp = []
test_tmp = []
max_count = 0
set_words = set()

print("Tokenizing!")
# test의 tmp만 따로 받아주고 나머지는 val train test모두 통합시켜준다.
test_tmp, dummy , set_words = tokenization(x_test)
val_tmp, dummy , set_words = tokenization(x_val)
train_tmp, max_count , set_words = tokenization(x_train)
tmp, dummy , set_words = tokenization(np.hstack([x_train,x_val,x_test]))
print("Tokenizing finished!")
tmp = [] # tmp없애준다



#token shape
print("It's by len()")
print("train_tmp.len():" + str(len(train_tmp))+","+ str(len(train_tmp[0])))
print("val_tmp shape.len():" + str(len(val_tmp))+","+ str(len(val_tmp[0])))
print("test_tmp shape.len():" + str(len(test_tmp))+","+ str(len(test_tmp[0])))





train_array(6651, 13)
val_array(887, 13)
test_array(3239, 13)
Reading data!
checking Anomaly!!
(6619,)
(6619, 11)
(883,)
(883, 11)
(3215,)
(3215, 11)
ANomaly result!
x_train.shape(6619,)
y_train.shape(6619, 11)
x_val.shape(883,)
y_val.shape(883, 11)
x_test.shape(3215,)
y_test.shape(3215, 11)
Finished!
Tokenizing!
Tokenizing finished!
It's by len()
train_tmp.len():6619,40
val_tmp shape.len():883,39
test_tmp shape.len():3215,46


In [94]:
type(x_val[0][0])

str

In [95]:

print("max_count:"+str(max_count))
# 문장길이 100으로 맞춘다.
max_count = min(100, max_count)


print("Readding Embedding file")
# embeddings_index == dict(w2v's word: vector)
f = codecs.open('/home/minwookje/coding/ex1_w2v/embedding/1542954106final_total_pos.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


629it [00:00, 6284.90it/s]

max_count:10531
Readding Embedding file


45942it [00:06, 7119.01it/s]

Loaded 45942 word vectors.


In [96]:

# word2index table 생성 {token:index}
# 모든 train val test에 사용되는 token별 index table
# train 문장 token 갯수
word_index = len(set_words)
word_tmp_dict = dict()
for i, word in enumerate(set_words):
    word_tmp_dict[str(word).replace(" ", "")] = i
word_tmp_dict['0'] = word_index 

## 문장별 토큰화시킨 녀석에 index를 집어 넣어준다. 이때 pad도 동시에 해준다. 
word_vec = []
word_vec_test = []
word_vec_val = []

for sent in train_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break    
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec.append(sub)
## 테스트용 복사본 
for sent in test_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec_test.append(sub)

## 검증용 복사본 
for sent in val_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec_val.append(sub)

print("word_vec shape:" + str(len(word_vec))+","+ str(len(word_vec[0])))
print("word_vec_val shape:" + str(len(word_vec_val))+","+ str(len(word_vec_val[0])))
print("word_vec_test shape:" + str(len(word_vec_test))+","+ str(len(word_vec_test[0])))



word_vec shape:6619,100
word_vec_val shape:883,100
word_vec_test shape:3215,100


In [97]:
print((type(word_vec_test[0][0])))

<class 'int'>


In [98]:


# 4번쨰 matrix embedding_matrix {index: vector}
# vocab_size = min(MAX_NB_WORDS, word_index)
vocab_size = word_index
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size+1, EMB_DIM))
match_count = 0
unmatch_count = 0
mins = []
maxs = []
for word, i in word_tmp_dict.items():
    # if word != '0':
    if (word in embeddings_index):
        match_count += 1
        
        # embedding_matrix[i] = np.zeros(300)
#         embedding_matrix[i] = embeddings_index[word]
        embedding_matrix[i] = embeddings_index[word].astype('float32')
#         embedding_matrix[i] = np.random.uniform(-1.0,1.0,300).astype('float64')
        for i in embedding_matrix[i]:
            if i <= -1:
                mins.append(i)
            elif i >= 1:
                maxs.append(i)    
                
#         embedding_matrix[i] = np.random.uniform(-0.25,0.25,300)
    else:
        unmatch_count += 1
        # embedding_matrix[i] = np.zeros(300)
        # embedding_matrix[i] = np.random.uniform(-0.25,0.25,300) ## used for OOV words
        embedding_matrix[i] = np.random.uniform(-1.0,1.0,300).astype('float32')
print("match:" + str(match_count))
print("unmatch:" + str(unmatch_count))
#     여기부터
# 1.앞서 단어당 벡터 테이블(v) // embeddings_index, {w2v_word: vector}
# train_word(str(word).replace(" ", "")) == embedding
# 2.train 단어별 index (v) // word_tmp_dict {train_word(str(word).replace(" ", "")):index}
# 3.sentence padding, sentence to index
# 4.index당 vector table (v) //embedding_matrix {index: vector} 이녀석을 embedding weight에 넣어주어야 한다. 

# 문장 = [index들 나열 ] 
# 즉 embedding_matrix로 index를 seq에 넣어준묹장들을 train에 넣어줘야한다. 

match:11300
unmatch:12627


In [99]:
# print(mins)
# print(maxs)
# np.array화
print("word_vec shape:" + str(len(word_vec))+","+ str(len(word_vec[0])))
print("word_vec_val shape:" + str(len(word_vec_val))+","+ str(len(word_vec_val[0])))
print("word_vec_test shape:" + str(len(word_vec_test))+","+ str(len(word_vec_test[0])))

word_vec = np.array(word_vec)
word_vec_val = np.array(word_vec_val)
word_vec_test = np.array(word_vec_test)
embedding_matrix = embedding_matrix
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

print("word_vec shape:" + str(word_vec.shape))
print("word_vec_val shape:" + str(word_vec_val.shape))
print("word_vec_test shape:" + str(word_vec_test.shape))
print("y_train shape:" + str(y_train.shape))
print("y_val shape:" + str(y_val.shape))
print("y_test shape:" + str(y_test.shape))
print("embedding_matrix shape:" + str(embedding_matrix.shape))


word_vec shape:6619,100
word_vec_val shape:883,100
word_vec_test shape:3215,100
word_vec shape:(6619, 100)
word_vec_val shape:(883, 100)
word_vec_test shape:(3215, 100)
y_train shape:(6619, 11)
y_val shape:(883, 11)
y_test shape:(3215, 11)
embedding_matrix shape:(23927, 300)


In [100]:
for k,v in embeddings_index.items():
    print(type(v[0]))
    break
    
type(embedding_matrix[word_vec[0][0]][0])
# type(y_train)

<class 'numpy.float32'>


numpy.float64

In [101]:

# def jaccard_distance_acc(y_true, y_pred):
#     intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
#     sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
#     jac = (intersection) / (sum_ - intersection)
#     return jac

INPUT_DIM = 300 #wordvec사이즈
max_count = max_count #문장의 max길이
lstm_shape = 250
rate_drop_lstm = 0.15
rate_drop_dense = 0.15

k_vec = word_vec
k_vec_val = word_vec_val
k_vec_test = word_vec_test
k_y_train = y_train
k_y_val = y_val
k_y_test = y_test
k_embedding_matrix = embedding_matrix

# nan delete
index = np.argwhere(np.isnan(k_y_train))[:,0]
index2 = np.argwhere(np.isnan(k_y_val))[:,0]
index3 = np.argwhere(np.isnan(k_y_test))[:,0]

k_y_train = np.delete(k_y_train,index,0)
k_vec = np.delete(k_vec,index,0)
k_y_val = np.delete(k_y_val,index2,0)
k_vec_val = np.delete(k_vec_val,index2,0)
k_vec_test = np.delete(k_vec_test,index3,0)
k_y_test = np.delete(k_y_test,index3,0)
# is nan check
print(np.any(np.isnan(k_vec)))
print(np.any(np.isnan(k_vec_val)))
print(np.any(np.isnan(k_vec_test)))
print(np.any(np.isnan(k_y_train)))
print(np.any(np.isnan(k_y_val)))
print(np.any(np.isnan(k_y_test)))
print(np.any(np.isnan(k_embedding_matrix)))

False
False
False
False
False
False
False


In [ ]:
from keras.layers import Activation, Concatenate, Permute, SpatialDropout1D, RepeatVector, LSTM, Bidirectional, Multiply, Lambda, Dense, Dropout, Input,Flatten,Embedding
from keras.models import Model
import keras.backend as K
import tensorflow as tf

from keras import backend as K

def jaccard_distance_loss(y_true, y_pred):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """

    epsilon = tf.convert_to_tensor(1e-7, dtype='float32')
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection) / (sum_ - intersection +epsilon)
    return (1 - jac)

# matrix accu
def jaccard_distance_acc(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection) / (sum_ - intersection)
    return jac
accu = jaccard_distance_acc    

# 32하니까 죽어버린다.
# 512
batch_size = 512
inp = Input(shape=(max_count,), dtype='float32')
emb = Embedding(input_dim=vocab_size+1, output_dim=EMB_DIM,
            trainable=False, weights=[embedding_matrix], input_length=max_count)(inp)
# max_features = vocab_size, maxlen=text_max_words, embed_size=EMB_DIM
# emb = Embedding(input_dim=max_features, input_length = maxlen, output_dim=embed_size)(inp)
# embedding dropout = 0.1
x = SpatialDropout1D(0.1)(emb)
x = Bidirectional(LSTM(lstm_shape, return_sequences=True, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))(x)
repeat_size = int(x.shape[-1])
x_a = Dense(repeat_size, kernel_initializer = 'glorot_uniform', activation="tanh", name="tanh_mlp")(x)
x_a= Dense(1, kernel_initializer = 'glorot_uniform', activation='linear', name="word-level_context")(x_a)
x_a= Flatten()(x_a)
att_out = Activation('softmax')(x_a) 
x_a2 = RepeatVector(repeat_size)(att_out)
x_a2 = Permute([2,1])(x_a2)
out = Multiply()([x,x_a2])
# x = Dense(11, activation="sigmoid")(x)
out = Lambda(lambda x : K.sum(x, axis=1), name='expectation_over_words')(out)
out = Dense(11, activation="sigmoid")(out)
model = Model(inputs=inp, outputs=out)


# model.compile(loss=jaccard_distance_loss,
#             optimizer='adam',
#             metrics=['accuracy'])
model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=[accu])
# intermediate_output = model.predict(k_vec)
model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=10, verbose=1, batch_size=batch_size)
# # model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=1, verbose=1,batch_size=None)


# # model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=3, verbose=1, steps_per_epoch=int(6619/batch_size)+1, validation_steps = int(883/batch_size)+1)

# # 5. 모델 평가하기
score, acc = model.evaluate(k_vec_test, k_y_test, batch_size=batch_size)
# # score, acc = model.evaluate(k_vec_test, k_y_test,batch_size=None)
print('')
print('score : ' + str(score)+ 'acc : ' + str(acc))


# # loss = tf.nn.sigmoid_cross_entropy_with_logits
# # loss = binary_crossentropy
# model.compile(loss='binary_crossentropy',
#             optimizer='adam',
#             metrics=['accuracy'])
# attention_model = Model(inputs=inp, outputs=attention) # Model to print out the attention data
model.summary()
# verbose= ? , validation_split은 validation file로 변환시켜주어야 한다.



# h = model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=1, verbose=1, batch_size=batch_size)
# # model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=1, verbose=1,batch_size=None)


# # model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=3, verbose=1, steps_per_epoch=int(6619/batch_size)+1, validation_steps = int(883/batch_size)+1)

# # 5. 모델 평가하기
# score, acc = model.evaluate(k_vec_test, k_y_test, batch_size=batch_size)
# # score, acc = model.evaluate(k_vec_test, k_y_test,batch_size=None)
# print('')
# print('score : ' + str(score)+ 'acc : ' + str(acc))

Train on 6474 samples, validate on 867 samples
Epoch 1/10
6474/6474 [==============================] - 110s 17ms/step - loss: 0.5135 - jaccard_distance_acc: 0.1627 - val_loss: 0.4818 - val_jaccard_distance_acc: 0.1514
Epoch 2/10
6474/6474 [==============================] - 110s 17ms/step - loss: 0.4730 - jaccard_distance_acc: 0.1610 - val_loss: 0.4803 - val_jaccard_distance_acc: 0.1563
Epoch 3/10
6474/6474 [==============================] - 120s 18ms/step - loss: 0.4713 - jaccard_distance_acc: 0.1608 - val_loss: 0.4754 - val_jaccard_distance_acc: 0.1636
Epoch 4/10
6474/6474 [==============================] - 119s 18ms/step - loss: 0.4705 - jaccard_distance_acc: 0.1609 - val_loss: 0.4745 - val_jaccard_distance_acc: 0.1701
Epoch 5/10
6474/6474 [==============================] - 112s 17ms/step - loss: 0.4704 - jaccard_distance_acc: 0.1619 - val_loss: 0.4743 - val_jaccard_distance_acc: 0.1675
Epoch 6/10
6474/6474 [==============================] - 104s 16ms/step - loss: 0.4702 - jaccard_di

In [23]:
# hidden_states = K.variable(value=np.random.normal(size=(100, 500)))
# cell_states = K.variable(value=np.random.normal(size=(100, 500)))

# model.layers[3].states[0] = hidden_states
# model.layers[3].states[1] = cell_states 

model = h.model  # include here your original model
k = np.array(list(k_vec))
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('bidirectional_5').get_input_at(0))
intermediate_output = intermediate_layer_model.predict(k)


# # with a Sequential model
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[1].output])
# layer_output = get_3rd_layer_output([x])[0]

In [40]:
from keras.layers import Activation, Concatenate, Permute, SpatialDropout1D, RepeatVector, LSTM, Bidirectional, Multiply, Lambda, Dense, Dropout, Input,Flatten,Embedding
from keras.models import Model
import keras.backend as K
# 32하니까 죽어버린다.
# 512
batch_size = 32
inp = Input(shape=(max_count,), dtype='float64')
emb = Embedding(input_dim=vocab_size+1, output_dim=EMB_DIM,
            trainable=False, weights=[embedding_matrix], input_length=max_count)(inp)
# max_features = vocab_size, maxlen=text_max_words, embed_size=EMB_DIM
# emb = Embedding(input_dim=max_features, input_length = maxlen, output_dim=embed_size)(inp)
# embedding dropout = 0.1
x = SpatialDropout1D(0.1)(emb)
x = Bidirectional(LSTM(lstm_shape, return_sequences=True, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))(x)
repeat_size = int(x.shape[-1])
x_a = Dense(repeat_size, kernel_initializer = 'glorot_uniform', activation="tanh", name="tanh_mlp")(x)
x_a= Dense(1, kernel_initializer = 'glorot_uniform', activation='linear', name="word-level_context")(x_a)
x_a= Flatten()(x_a)
att_out = Activation('softmax')(x_a) 
x_a2 = RepeatVector(repeat_size)(att_out)
x_a2 = Permute([2,1])(x_a2)
out = Multiply()([x,x_a2])
# x = Dense(11, activation="sigmoid")(x)
out = Lambda(lambda x : K.sum(x, axis=1), name='expectation_over_words')(out)
out = Dense(11, activation="sigmoid")(out)
model = Model(inputs=inp, outputs=out)


# model.compile(loss='binary_crossentropy',
#             optimizer='adam',
#             metrics=['accuracy'])
intermediate_output = model.predict(k_vec)
# h = model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=1, verbose=1, batch_size=batch_size)
# # model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=1, verbose=1,batch_size=None)


# # model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=3, verbose=1, steps_per_epoch=int(6619/batch_size)+1, validation_steps = int(883/batch_size)+1)

# # 5. 모델 평가하기
# score, acc = model.evaluate(k_vec_test, k_y_test, batch_size=batch_size)
# # score, acc = model.evaluate(k_vec_test, k_y_test,batch_size=None)
# print('')
# print('score : ' + str(score)+ 'acc : ' + str(acc))


# # loss = tf.nn.sigmoid_cross_entropy_with_logits
# # loss = binary_crossentropy
# model.compile(loss='binary_crossentropy',
#             optimizer='adam',
#             metrics=['accuracy'])
# attention_model = Model(inputs=inp, outputs=attention) # Model to print out the attention data
model.summary()
# verbose= ? , validation_split은 validation file로 변환시켜주어야 한다.



# h = model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=1, verbose=1, batch_size=batch_size)
# # model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=1, verbose=1,batch_size=None)


# # model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=3, verbose=1, steps_per_epoch=int(6619/batch_size)+1, validation_steps = int(883/batch_size)+1)

# # 5. 모델 평가하기
# score, acc = model.evaluate(k_vec_test, k_y_test, batch_size=batch_size)
# # score, acc = model.evaluate(k_vec_test, k_y_test,batch_size=None)
# print('')
# print('score : ' + str(score)+ 'acc : ' + str(acc))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 100, 300)     7178100     input_11[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_11 (SpatialDr (None, 100, 300)     0           embedding_11[0][0]               
__________________________________________________________________________________________________
bidirectional_11 (Bidirectional (None, 100, 500)     1102000     spatial_dropout1d_11[0][0]       
__________________________________________________________________________________________________
tanh_mlp (

In [54]:
# for i in intermediate_output:
#     print(i)
# type(intermediate_output[0][0])
type(k_y_train[0][0])

numpy.float32

In [61]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.framework import ops as tf_ops
from tensorflow.python.training import moving_averages
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import functional_ops
from tensorflow.python.ops import ctc_ops as ctc
from tensorflow.python.client import device_lib
from tensorflow.core.protobuf import config_pb2

from collections import defaultdict

import numpy as np
from distutils.version import StrictVersion
import os

# from .common import floatx
# from .common import epsilon
# from .common import normalize_data_format
# from ..utils.generic_utils import transpose_shape
# from ..utils.generic_utils import has_arg

# # Legacy functions
# from .common import set_image_dim_ordering
# from .common import image_dim_ordering
# epsilon()

def _to_tensor(x, dtype):
    """Convert the input `x` to a tensor of type `dtype`.
    # Arguments
        x: An object to be converted (numpy array, list, tensors).
        dtype: The destination type.
    # Returns
        A tensor.
    """
    return tf.convert_to_tensor(x, dtype=dtype)

def binary_crossentropy(target, output, from_logits=False):
    """Binary crossentropy between an output tensor and a target tensor.
    # Arguments
        target: A tensor with the same shape as `output`.
        output: A tensor.
        from_logits: Whether `output` is expected to be a logits tensor.
            By default, we consider that `output`
            encodes a probability distribution.
    # Returns
        A tensor.
    """
    # Note: tf.nn.sigmoid_cross_entropy_with_logits
    # expects logits, Keras expects probabilities.
    if not from_logits:
        # transform back to logits
        _epsilon = _to_tensor(1e-3, 'float32')
        output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
        output = tf.log(output / (1 - output))

    return tf.nn.sigmoid_cross_entropy_with_logits(labels=target,
                                                   logits=output)
sess = tf.Session()
for i in range(len(intermediate_output)):
    print(str(i)+"번째 binary값")
    print(sess.run(binary_crossentropy(intermediate_output[i],k_y_train[i])))

0번째 binary값
[ 3.56274462  3.54656816  3.17959499  4.14543724  3.65040946  3.40665507
  3.39636278  3.87953687  3.49503088  3.04400015  3.73767209]
1번째 binary값
[ 3.50357461  3.29399228  3.0648303   4.22947741  3.22625685  3.60017586
  3.347965    4.01286411  3.58111811  3.00994802  3.0973928 ]
2번째 binary값
[ 3.32466125  3.50519013  3.46845579  3.96044707  3.30048609  3.43269682
  3.53478909  3.61211395  3.6979661   3.05185151  3.31565237]
3번째 binary값
[ 3.46939039  3.39872599  3.23184872  4.1199913   3.17656398  3.33269644
  3.44089484  3.93464756  3.58289242  3.04069376  3.18482375]
4번째 binary값
[ 3.39366627  3.41134953  3.70755839  4.1231122   3.77722955  3.37591076
  3.52570701  3.92815566  3.57129025  2.95168853  3.14350152]
5번째 binary값
[ 3.57131648  3.40438318  3.17216611  4.07928133  3.20399594  3.32768655
  3.49779844  3.81893468  3.56745791  3.04154682  3.18881297]
6번째 binary값
[ 3.45941973  3.62555122  3.06791711  4.17387772  3.69996285  3.35238695
  3.55867124  2.91193724  3.58656

KeyboardInterrupt: 

In [50]:
# print(type(intermediate_output[0][0]))
# print(k_y_train[0])
# print(intermediate_output[0])
sess = tf.Session()
for i in range(len(intermediate_output)):
    a = jaccard_distance_loss(k_y_train[i],intermediate_output[i])
    print(str(i)+"번째 loss값")
    print(sess.run(a))

0번째 loss값
0.770278
1번째 loss값
0.801499
2번째 loss값
0.744164
3번째 loss값
0.869105
4번째 loss값
0.851815
5번째 loss값
0.924549
6번째 loss값
0.84073
7번째 loss값
0.841391
8번째 loss값
0.85332
9번째 loss값
0.849299
10번째 loss값
0.797735
11번째 loss값
1.0
12번째 loss값
0.84415
13번째 loss값
0.853868
14번째 loss값
0.85255
15번째 loss값
0.795877
16번째 loss값
0.795859
17번째 loss값
0.761163
18번째 loss값
0.872157
19번째 loss값
0.785898
20번째 loss값
0.850252
21번째 loss값
0.852842
22번째 loss값
0.802451
23번째 loss값
0.843368
24번째 loss값
0.74084
25번째 loss값
0.862723
26번째 loss값
0.874518
27번째 loss값
0.79732
28번째 loss값
0.745591
29번째 loss값
0.856394
30번째 loss값
0.740749
31번째 loss값
0.779374
32번째 loss값
0.797959
33번째 loss값
0.781422
34번째 loss값
0.803129
35번째 loss값
0.839571
36번째 loss값
0.872567
37번째 loss값
0.928037
38번째 loss값
1.0
39번째 loss값
0.722814
40번째 loss값
0.755334
41번째 loss값
0.863876
42번째 loss값
0.914002
43번째 loss값
0.840422
44번째 loss값
0.744093
45번째 loss값
0.791272
46번째 loss값
0.804088
47번째 loss값
0.790111
48번째 loss값
0.774096
49번째 loss값
1.0
50번째 loss값
0.79464
51번째 loss값
0

KeyboardInterrupt: 

In [52]:
print(intermediate_output[185])
print(k_y_train[185])

[ 0.45663142  0.55807221  0.50074631  0.52394301  0.44589725  0.53704381
  0.39955401  0.45307899  0.45444661  0.43341556  0.60899854]
[  0.   0.   0.   0.   0.   0.   1.   1.   0.   0.  nan]


In [ ]:
test = tf.convert_to_tensor(np.array([0.50548059,0.50548059,0,0,0,0,1,0,0,0,1]), dtype='float64')

In [38]:

a = jaccard_distance_loss(k_y_train[0],intermediate_output[0])
sess = tf.Session()
sess.run(a)

0.79676378

In [43]:
b = jaccard_distance_loss(k_y_train[0],test)
sess = tf.Session()
sess.run(b)

TypeError: Input 'y' of 'Add' Op has type float64 that does not match type float32 of argument 'x'.

In [63]:
final_x = list()
final_y = list()
k = [[1,2,3,4,'nan'],[1,2,7,4,5],[1,2,3,4,5]]
for i,y in  np.ndenumerate(np.array(k)):
#     print(i[0],y)
    for j in y:
        if (np.isnan(j)==True):
            print(y)
#             final_x.append([1])
#             final_y.append([2])


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [60]:
numk=np.array(k)

In [66]:
import numpy as np
a = np.array([[np.NaN,np.NaN,np.NaN,np.NaN],[np.NaN,np.NaN,np.NaN,np.NaN]])
print(np.isnan(a))
print(np.any(np.isnan(a)))

[[ True  True  True  True]
 [ True  True  True  True]]
True


In [67]:
np.any(np.isnan(k_y_train))

True

In [70]:
print(np.any(np.isnan(k_vec)))
print(np.any(np.isnan(k_vec_val)))
print(np.any(np.isnan(k_vec_test)))
print(np.any(np.isnan(k_y_train)))
print(np.any(np.isnan(k_y_val)))
print(np.any(np.isnan(k_y_test)))
print(np.any(np.isnan(k_embedding_matrix)))
k_vec = word_vec
k_vec_val = word_vec_val
k_vec_test = word_vec_test
# k_y_train = y_train
# k_y_val = y_val
# k_y_test = y_test
# k_embedding_matrix = embedding_matrix


False
False
False
True
True
True
False


In [71]:
ttt = k_y_train[~np.isnan(k_y_train).any(axis=1)]
print(np.any(np.isnan(ttt)))

False


In [77]:
index = np.argwhere(np.isnan(k_y_val))[:][0]
print(index)

[[ 63  10]
 [112  10]
 [189  10]
 [266  10]
 [301  10]
 [350  10]
 [390  10]
 [477  10]
 [584  10]
 [585  10]
 [624  10]
 [663   0]
 [687  10]
 [744  10]
 [773  10]
 [848  10]]


In [85]:
x = np.array([[np.nan,5,2,3],[1,2,3,4],
              [2,3,np.nan,5],
              [np.nan,5,2,3],[np.nan,5,2,3]])

In [91]:
index = np.argwhere(np.isnan(k_y_train))[:,0]
index2 = np.argwhere(np.isnan(k_y_val))[:,0]
index3 = np.argwhere(np.isnan(k_y_test))[:,0]

k_y_train = np.delete(k_y_train,index,0)
k_vec = np.delete(k_vec,index,0)
k_y_val = np.delete(k_y_val,index2,0)
k_vec_val = np.delete(k_vec_val,index2,0)
k_vec_test = np.delete(k_vec_test,index3,0)
k_y_test = np.delete(k_y_test,index3,0)
    

In [92]:
print(np.any(np.isnan(k_vec)))
print(np.any(np.isnan(k_vec_val)))
print(np.any(np.isnan(k_vec_test)))
print(np.any(np.isnan(k_y_train)))
print(np.any(np.isnan(k_y_val)))
print(np.any(np.isnan(k_y_test)))
print(np.any(np.isnan(k_embedding_matrix)))

False
False
False
False
False
False
False


In [90]:
arr = np.array([[1,1,1,1],[2,2,2,4],[3,3,3,4], [5,6,7,8], [9,10,11,12]])
print(np.delete(arr,index,0))

[[2 2 2 4]]
